In [1]:
import numpy as np
import scipy
import sympy
import data_import
import pandas as pd
from matplotlib import pyplot as plt

# Data Import & Pre-processing

In [2]:
pd_data = data_import.read_files_to_pd_dataframe(
    [f"../outs/grid/itr_{i}.xml.out" for i in range(500)]
)

# Policy Iteration Algo

In [3]:

def get_policy(Q, x, b):
    """
    updates policy to the next policy,
    returns error
    """
    policy = []
    err2 = 0
    y = Q @ x + b
    for i in range(len(x)):
        if y[i] < x[i]:
            policy.append(i)
        err2 += min(x[i], y[i]) ** 2    # x should be > 0, while y should be as close to 0 as possible
    return err2 ** 0.5, policy

# get the sub-system of linear equations given the policy
def get_sub_sys(policy, Q, b):
    policy_matrix = np.zeros((b.shape[0], b.shape[0]))
    for p in policy:
        policy_matrix[p,p] = 1
    I = np.eye(len(b))
    A = np.dot(
            np.dot(
                policy_matrix,
                Q
            ),
            policy_matrix
        ) + I - policy_matrix

    rhs = - policy_matrix @ b 
    # new_Q = np.zeros((len(policy), len(policy)))
    # new_b = np.zeros(len(policy))

    # for new_row, old_row in enumerate(policy):
    #     new_b[new_row] = b[old_row]
    #     for new_col, old_col in enumerate(policy):
    #         new_Q[new_row, new_col] = Q[old_row, old_col]
    
    return A, rhs
    


def update_value(policy, Q, b, use_cg = True):
    """
    returns the next x given the policy
    """
    I = np.eye(len(b))
    policy_matrix = np.zeros((b.shape[0], b.shape[0]))
    for p in policy:
        policy_matrix[p,p] = 1
    A = policy_matrix @ Q @ policy_matrix + I - policy_matrix
    rhs = - policy_matrix @ b 
    lst_sqr_soln = np.linalg.lstsq(A , rhs)[0]
    cg_soln = scipy.sparse.linalg.cg(A , rhs , tol=1e-12)[0]
    if use_cg:
        return cg_soln
    else:
        return lst_sqr_soln


def flow(Q, N, v0, correct_policy, max_itrs = 100, m_tol = 1e-09, CoR = 1, use_cg = True):
    """
    does policy iteration
    returns a bool/vector pair: (converged, x)
    """
    b = np.dot(N.T, (1 + CoR) * v0)
    x = np.zeros(b.shape[0])
    policy = []
    error = 0.0

    found_correct_control = False

    for n_iter in range(max_itrs):

        # have we stumbled across the correct policy/control?
        if correct_policy == policy:
            found_correct_control = True

        error, policy = get_policy(Q, x, b)
        if error <= m_tol:
            return (True, x, found_correct_control)

        x = update_value(policy, Q, b, use_cg)
    
    return (False, x, found_correct_control)



# For sim 181...

policy iteration ends up hitting the "correct" policy (i.e. the policy that could feasibly result in the ipopt soln),
but the inner solver doesn't give a "good" soln, so we move off of the "correct" policy to a "worse?" one

In [4]:
def get_ipopt_control(x):
    # anything less than 1e-6 is counted as a 0 since thats the tolerance we set in the xml files
    return [i for i in range(len(x)) if x[i] > 1e-6]

pd_data['correct_control'] = [get_ipopt_control(x) for x in pd_data['ipopt_sol']]
# pd_data['correct_control']

In [5]:
i = 181
Q = pd_data['Q'][i]
N = pd_data['N'][i]
v0 = pd_data['v0'][i]
M = np.linalg.inv(pd_data['Minv'][i])
Minv = pd_data['Minv'][i]
b = np.dot(N.T, (1 + 1) * v0)

correct_control = pd_data['correct_control'][i]

sub_Q, rhs = get_sub_sys(correct_control, Q, b)

np.set_printoptions(linewidth=1000, precision=6)
# scipy.linalg.null_space(sub_Q, 1e-5)  # null space is empty!
cg_soln = scipy.sparse.linalg.cg(sub_Q , rhs , tol=1e-12)[0]

ipopt_soln = pd_data['ipopt_sol'][i]


# print(np.around(Q @ ipopt_soln + b, 5))

# print(get_policy(Q, ipopt_soln, b))

# change in kinetic energy?
p = M @ v0

ke_0 = 0.5 * p.T @ Minv @ p
ke_1 = 0.5 * (N @ ipopt_soln + p).T @ Minv @ (N @ ipopt_soln + p)
print(ke_0)
print(ke_1)
print(abs(ke_0 - ke_1)/ke_1)



87.01410191379713
87.0137689991097
3.82600008323945e-06


it turns out that not even the ipopt solution is the "correct" solution <- incorrect!

In [6]:
def get_obj_err(N, sol, p, Minv):
    return abs(
        0.5 * (N @ sol + p).T @ Minv @ (N @ sol + p)
        - 0.5 * p.T @ Minv @ p
    ) / 0.5 * p.T @ Minv @ p

In [7]:
errs = []
obj_errs = []
obj_errs_ipopt = np.zeros(500)
for i in range(500):

    Q = pd_data['Q'][i]
    N = pd_data['N'][i]
    v0 = pd_data['v0'][i]
    b = np.dot(N.T, (1 + 1) * v0)
    Minv = pd_data['Minv'][i]
    M = np.linalg.inv(Minv)
    p = M @ v0
    policy = pd_data['correct_control'][i]
    
    this_pi_soln = update_value(policy, Q, b)

    this_pi_soln_err, _ = get_policy(Q, this_pi_soln, b)
    errs.append(this_pi_soln_err)
    obj_errs.append(get_obj_err(N, this_pi_soln, p, Minv))

    obj_errs_ipopt[i] = get_obj_err(N, pd_data['policy_sol'][i], p, Minv)

errs = np.array(errs)
# print(scipy.stats.describe(errs))
# plt.hist(errs, bins=100)
obj_errs = np.array(obj_errs)
np.count_nonzero(obj_errs[obj_errs <= 1e+01])
# print(scipy.stats.describe(errs))
# plt.hist(obj_errs_ipopt,bins=100)
# print(np.count_nonzero(obj_errs_ipopt > 1e-1))
print([i for i in range(500) if obj_errs_ipopt[i] > 1])

# print([i for i in range(500) if obj_errs[i] > 1 and errs[i] > 1])
# print([i for i in range(500) if obj_errs[i] > 1])
# print([i for i in range(500) if errs[i] > 1])
# print([i for i in range(500) if obj_errs[i] > 1])




/tmp/ipykernel_24663/2829095354.py:58: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  lst_sqr_soln = np.linalg.lstsq(A , rhs)[0]


[124, 420]


# computing the delta kinetic energy for the ipopt solutions.

If it's actually giving an incorrect answer, dH should be non-0

(see `data_import.py`)

In [11]:
pd_data['ipopt_delta_KE']

0      0.000012
1      0.000015
2      0.000034
3      0.000049
4      0.000023
         ...   
495    0.000072
496    0.000169
497    0.000093
498    0.000064
499    0.000365
Name: ipopt_delta_KE, Length: 500, dtype: float64

all these seem ok... max error was about 0.0005... i think this is reasonable?